In [78]:
import pandas as pd
import numpy as np
import datetime as dt
import requests 
import json

In [79]:
# 0001318605

In [80]:
url = 'https://data.sec.gov/api/xbrl/companyfacts/CIK0001318605.json'
ticker = 'https://www.sec.gov/files/company_tickers.json'

In [81]:
headers ={
    'User-Agent': 'YOUR_USER_AGENT@gmail.com',
    'Accept-Encoding': 'gzip, deflate',
    'Host':'data.sec.gov'
    }

In [82]:
r = requests.get(url, headers = headers)
tik = requests.get(ticker)

In [83]:
print('API Request Server:' ,r.status_code)
print('Ticker table Server:',tik.status_code)

API Request Server: 200
Ticker table Server: 200


In [84]:
ticker = pd.DataFrame(tik.json())

In [85]:
Tickers = pd.DataFrame({
'cik_str' : ticker.iloc[0],
'ticker': ticker.iloc[1],
'title': ticker.iloc[2]
})

In [86]:
Tickers['CIK'] = Tickers['cik_str'].astype(str).str.zfill(10)

In [87]:
Tickers.head()

,cik_str,ticker,title,CIK
0,320193,AAPL,Apple Inc.,0000320193
1,789019,MSFT,MICROSOFT CORP,0000789019
2,1318605,TSLA,"Tesla, Inc.",0001318605
3,1067983,BRK-B,BERKSHIRE HATHAWAY INC,0001067983
4,731766,UNH,UNITEDHEALTH GROUP INC,0000731766


In [88]:
My_stock = "TSLA"
Tickers.loc[Tickers['ticker'] == My_stock]['CIK']

2    0001318605
Name: CIK, dtype: object

In [89]:
data = r.json()

In [90]:
dei_data = [] 
for k in data['facts']['dei']:
    for i in data['facts']['dei'][k]['units']:
        for j in data['facts']['dei'][k]['units'][i]:
            d_data = {
                'account': k,
                'end': j['end'],
                'val': j['val'],
                'accn': j['accn'],
                'fy': j['fy'],
                'fp': j['fp'],
                'form': j['form'],
                'filed': j['filed'],
                'frame': j.get('frame', 'NULL')
                ,
            }
            dei_data.append(d_data)

In [91]:
dei = pd.DataFrame(dei_data)

In [92]:
dei

,account,end,val,accn,fy,fp,form,filed,frame
0,EntityCommonStockSharesOutstanding,2011-07-31,104018274,0001193125-11-221497,2011,Q2,10-Q,2011-08-12,CY2011Q2I
1,EntityCommonStockSharesOutstanding,2011-10-31,104298634,0001193125-11-308489,2011,Q3,10-Q,2011-11-14,CY2011Q3I
2,EntityCommonStockSharesOutstanding,2012-01-31,104604044,0001193125-12-081990,2011,FY,10-K,2012-02-27,NULL
3,EntityCommonStockSharesOutstanding,2012-01-31,104604044,0001193125-12-137560,2011,FY,10-K/A,2012-03-28,CY2011Q4I
4,EntityCommonStockSharesOutstanding,2012-04-30,105214400,0001193125-12-225825,2012,Q1,10-Q,2012-05-10,CY2012Q1I
...,...,...,...,...,...,...,...,...,...
59,EntityPublicFloat,2019-06-30,31540000000,0001564590-20-018984,2019,FY,10-K/A,2020-04-28,CY2019Q2I
60,EntityPublicFloat,2020-06-30,160570000000,0001564590-21-004599,2020,FY,10-K,2021-02-08,NULL
61,EntityPublicFloat,2020-06-30,160570000000,0001564590-21-022604,2020,FY,10-K/A,2021-04-30,CY2020Q2I
62,EntityPublicFloat,2021-06-30,541280000000,0000950170-22-000796,2021,FY,10-K,2022-02-07,NULL


# GAAP

In [93]:
pandas_data = []
for k in data['facts']['us-gaap']:
    for i in data['facts']['us-gaap'][k]['units']:
        for j in data['facts']['us-gaap'][k]['units'][i]:
            val_data = {
                'account': k,
                'end': j['end'],
                'val': j['val'],
                'accn': j['accn'],
                'fy': j['fy'],
                'fp': j['fp'],
                'form': j['form'],
                'filed': j['filed'],
                'frame': j.get('frame', 'NULL')
                # 'frame': j['frame'],
            }
            pandas_data.append(val_data)
    

In [94]:
data_frame = pd.DataFrame(pandas_data)

In [95]:
data_frame.drop_duplicates(inplace = True)

In [96]:
#my custom deal account for filtering and grouping. 
group_deal = (data_frame['account'] != data_frame['account'].shift())
data_frame['deal_group'] = np.cumsum(group_deal)

In [97]:
data_frame

,account,end,val,accn,fy,fp,form,filed,frame,deal_group
0,AccountsAndNotesReceivableNet,2019-12-31,402000000.0,0001564590-20-047486,2020,Q3,10-Q,2020-10-26,NULL,1
1,AccountsAndNotesReceivableNet,2019-12-31,402000000.0,0001564590-21-004599,2020,FY,10-K,2021-02-08,CY2019Q4I,1
2,AccountsAndNotesReceivableNet,2020-09-30,342000000.0,0001564590-20-047486,2020,Q3,10-Q,2020-10-26,CY2020Q3I,1
3,AccountsAndNotesReceivableNet,2020-12-31,334000000.0,0001564590-21-004599,2020,FY,10-K,2021-02-08,NULL,1
4,AccountsAndNotesReceivableNet,2020-12-31,334000000.0,0000950170-21-000046,2021,Q1,10-Q,2021-04-28,NULL,1
...,...,...,...,...,...,...,...,...,...,...
17210,MarketableSecurities,2021-12-31,131000000.0,0000950170-22-012936,2022,Q2,10-Q,2022-07-25,CY2021Q4I,573
17211,MarketableSecurities,2022-03-31,508000000.0,0000950170-22-006034,2022,Q1,10-Q,2022-04-25,CY2022Q1I,573
17212,MarketableSecurities,2022-06-30,591000000.0,0000950170-22-012936,2022,Q2,10-Q,2022-07-25,CY2022Q2I,573
17213,AntidilutiveSecuritiesExcludedFromComputationO...,2022-06-30,1000000.0,0000950170-22-012936,2022,Q2,10-Q,2022-07-25,NULL,574


In [98]:
data_frame

,account,end,val,accn,fy,fp,form,filed,frame,deal_group
0,AccountsAndNotesReceivableNet,2019-12-31,402000000.0,0001564590-20-047486,2020,Q3,10-Q,2020-10-26,NULL,1
1,AccountsAndNotesReceivableNet,2019-12-31,402000000.0,0001564590-21-004599,2020,FY,10-K,2021-02-08,CY2019Q4I,1
2,AccountsAndNotesReceivableNet,2020-09-30,342000000.0,0001564590-20-047486,2020,Q3,10-Q,2020-10-26,CY2020Q3I,1
3,AccountsAndNotesReceivableNet,2020-12-31,334000000.0,0001564590-21-004599,2020,FY,10-K,2021-02-08,NULL,1
4,AccountsAndNotesReceivableNet,2020-12-31,334000000.0,0000950170-21-000046,2021,Q1,10-Q,2021-04-28,NULL,1
...,...,...,...,...,...,...,...,...,...,...
17210,MarketableSecurities,2021-12-31,131000000.0,0000950170-22-012936,2022,Q2,10-Q,2022-07-25,CY2021Q4I,573
17211,MarketableSecurities,2022-03-31,508000000.0,0000950170-22-006034,2022,Q1,10-Q,2022-04-25,CY2022Q1I,573
17212,MarketableSecurities,2022-06-30,591000000.0,0000950170-22-012936,2022,Q2,10-Q,2022-07-25,CY2022Q2I,573
17213,AntidilutiveSecuritiesExcludedFromComputationO...,2022-06-30,1000000.0,0000950170-22-012936,2022,Q2,10-Q,2022-07-25,NULL,574


In [99]:
#export to excel
data_frame.to_excel(f'{My_stock}.xlsx')

In [100]:
# How many file forms? 
pd.DataFrame(data_frame['form'].value_counts())

,form
10-Q,10789
10-K,5885
10-K/A,536


In [110]:
pd.DataFrame(data_frame['fy'].value_counts()).sort_index()

,fy
2011,1148
2012,1441
2013,1369
2014,1437
2015,1314
2016,1345
2017,1529
2018,1607
2019,1854
2020,1694
